In [117]:
%pip install pymupdf

Defaulting to user installation because normal site-packages is not writeable
  Using cached s3transfer-0.6.2-py3-none-any.whl (79 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 867.8 kB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.65
    Uninstalling botocore-1.31.65:
      Successfully uninstalled botocore-1.31.65
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.7.0
    Uninstalling s3transfer-0.7.0:
      Successfully uninstalled s3transfer-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.65 requires botocore==1.31.65, but you have botocore 1.29.165 which is incompatible.
awscli 1.29.65 requires s3transfer<0.8.0,>=0.7.0, but you have s3transfer 0.6.2 which is incompatible.
Note: you may need to restart the kernel to use updated

# Processing OAB

## Answers

In [3]:
import pandas as pd
import sys, pathlib, fitz
from fitz import Rect
import json

ANSWERS_PDF_PATHS = [
    "1A FASE OAB/XXXVII EXAME UNIFICADO – GABARITOS.pdf",
]
def get_answers_dict(pdf_path):
    doc = fitz.open(pdf_path)
    page = doc[0]  # first page
    words = page.get_text("words", sort=True)  # extract sorted words
    start_idx = -1
    end_idx = -1
    for idx, w in enumerate(words):
        if "BRANCO" in w[4]:
            start_idx = idx
        elif "XXXVII" in w[4] and start_idx != -1 and end_idx == -1:
            end_idx = idx
            break

    answers_dict = {}
    question_idx = 1
    for word in (words[start_idx+1:end_idx]):
        if not word[4].isdigit():
            answers_dict[question_idx] = word[4]
            question_idx += 1
            
    parsed_answers_dict = {
        question_number : correct_ans 
        for question_number, correct_ans in answers_dict.items() 
        if question_number <= 80
        }
    json_object = json.dumps(parsed_answers_dict, indent=4)

    # Writing to sample.json
    with open(f"{pdf_path.replace('.pdf','')}.json", "w") as outfile:
        outfile.write(json_object)
    return parsed_answers_dict
for path in ANSWERS_PDF_PATHS:
    print(path)
    answers_dict = get_answers_dict(path)
    print(answers_dict)

    

1A FASE OAB/XXXV EXAME UNIFICADO - GABARITOS.pdf
{1: 'C', 2: 'D', 3: 'B', 4: 'D', 5: 'C', 6: 'C', 7: 'B', 8: 'A', 9: 'B', 10: 'B', 11: 'B', 12: 'B', 13: 'B', 14: 'D', 15: 'C', 16: 'A', 17: 'B', 18: 'C', 19: 'B', 20: 'B', 21: 'C', 22: 'C', 23: 'B', 24: 'C', 25: 'C', 26: 'A', 27: 'B', 28: 'A', 29: 'A', 30: 'C', 31: 'A', 32: 'B', 33: 'B', 34: 'D', 35: 'D', 36: 'C', 37: 'D', 38: 'D', 39: 'D', 40: 'B', 41: 'C', 42: 'C', 43: 'A', 44: 'D', 45: 'B', 46: 'C', 47: 'D', 48: 'D', 49: 'A', 50: '*', 51: 'A', 52: 'B', 53: 'C', 54: 'C', 55: 'D', 56: 'A', 57: 'C', 58: 'C', 59: '*', 60: 'D', 61: 'A', 62: 'A', 63: 'C', 64: 'C', 65: 'B', 66: 'D', 67: 'D', 68: 'B', 69: 'D', 70: 'C', 71: 'C', 72: 'B', 73: 'B', 74: 'B', 75: 'A', 76: 'A', 77: 'A', 78: 'A', 79: 'A', 80: 'C'}
1A FASE OAB/XXXVI EXAME UNIFICADO - GABARITOS.pdf
{1: 'B', 2: 'A', 3: 'D', 4: 'D', 5: 'B', 6: 'C', 7: 'D', 8: 'D', 9: 'A', 10: 'B', 11: 'C', 12: 'B', 13: 'D', 14: 'B', 15: 'C', 16: 'A', 17: 'A', 18: 'A', 19: 'D', 20: 'B', 21: 'D', 22: 'C',

## Questions

## Vade Mecum

In [4]:

vm_chapter_metadata = pd.read_csv("vade_mecum_chapter_metadata.csv")
vm_chapter_metadata.loc[((vm_chapter_metadata["start_page"] <= 10) & (vm_chapter_metadata["end_page"] >= 10)) ,"chapter_title"].values[0]
fname = "Vade_mecum_2023.pdf"  # get document filename

text_dict = {}
with fitz.open(fname) as doc:  # open document
    for idx, page in enumerate(doc):
        if idx+1 >= 17 and idx+1 <=885:
            chapter_title = vm_chapter_metadata.loc[((vm_chapter_metadata["start_page"] <= idx) & (vm_chapter_metadata["end_page"] >= idx)) ,"chapter_title"].values[0]
            text_dict[idx+1] = {"text" : ""}
            page_1st_third = page.get_text(clip=Rect([0, 0, 190, 1000]))
            page_2nd_third = page.get_text(clip=Rect([190, 0, 320, 1000]))
            page_3rd_third = page.get_text(clip=Rect([320, 0, 410, 1000]))
            text_dict[idx+1]["text"] = (
                page_1st_third+
                page_2nd_third+
                page_3rd_third
            )
            text_dict[idx+1]["chapter_title"] = chapter_title
with open(f'{fname}.json', 'w') as f:
    json.dump(text_dict, f)